In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns

In [6]:
pd.set_option('display.max_columns',None, 'display.max_rows',20)

In [101]:
file_path = os.path.join('../../../','processed_data/')
in_output_path = os.path.join('../../../','processed_data/provider_final.csv.csv')
os.listdir(file_path)

['.ipynb_checkpoints',
 'File_Description.txt',
 'Merge_final.csv',
 'provider_final.csv',
 'provider_final_old.csv']

In [103]:
df = pd.read_csv(os.path.join(file_path,'provider_final.csv'))
df = df.drop(df.columns[0],axis=1)
df.head()

,Provider,PotentialFraud,DurationMedianIP,DurationMeanIP,DurationMedianOP,DurationMeanOP,Serves,ChronCondsMedian,ChronCondsMean,ChronConsMode,PtAgeMedian,PtAgeMean,NumPtsIP,NumPtsOP,NumStatesIP,NumStatesOP,SharePhysician,MergeAvgClaimReimbursed,NumTopCodeClaim,DupClaimNum_IP,DupClaimRatio_IP,DupClaim_IP,DupClaimStatesNum_IP,DupClaimMultiState_IP,DupClaimProviderNum_IP,DupClaimMultiProvider_IP,DupClaimStartDtNum_IP,DupClaimMultiStartDt_IP,DupClaimNum_OP,DupClaimRatio_OP,DupClaim_OP,DupClaimStatesNum_OP,DupClaimMultiState_OP,DupClaimProviderNum_OP,DupClaimMultiProvider_OP,DupClaimStartDtNum_OP,DupClaimMultiStartDt_OP,AvgDuration,NumOfPt,NumOfClm,ClmPerPt,TotalRev,RevPerPt,ClmNoPhy,ClmAllPhy,ClmNoProc,ClmPerDt
0,PRV51001,No,4.0,5.000000,0.0,0.550000,Both,6.0,5.760000,5.0,79.0,77.880000,5.0,19.0,1.0,1.0,1.0,4185.600000,5.0,0.0,0.000000,no,0.0,no,0.0,no,0.0,no,11.0,0.550000,yes,50.0,yes,422.0,yes,278.0,yes,1.44,24,25,1.04,109980,4582.50,0,2,23,1.09
1,PRV51003,Yes,4.0,5.161290,0.0,2.357143,Both,7.0,6.674242,6.0,71.0,69.083333,53.0,66.0,3.0,3.0,1.0,4588.409091,25.0,1.0,0.016129,yes,2.0,yes,2.0,yes,2.0,yes,25.0,0.357143,yes,52.0,yes,1351.0,yes,372.0,yes,3.67,117,132,1.13,671956,5743.21,0,4,93,1.19
2,PRV51004,No,0.0,0.000000,0.0,1.429530,OP,7.0,6.812081,8.0,72.0,71.261745,0.0,138.0,0.0,9.0,1.0,350.134228,5.0,0.0,0.000000,no,0.0,no,0.0,no,0.0,no,72.0,0.483221,yes,52.0,yes,3006.0,yes,375.0,yes,1.43,138,149,1.08,52480,380.29,0,12,149,1.27
3,PRV51005,Yes,0.0,0.000000,0.0,1.088412,OP,7.0,6.886695,5.0,70.0,69.526180,0.0,495.0,0.0,4.0,1.0,241.124464,25.0,0.0,0.000000,no,0.0,no,0.0,no,0.0,no,530.0,0.454936,yes,52.0,yes,3920.0,yes,382.0,yes,1.09,495,1165,2.35,284610,574.97,2,98,1165,3.27
4,PRV51007,No,5.0,5.333333,0.0,0.768116,Both,7.0,6.986111,6.0,69.5,68.305556,3.0,56.0,1.0,2.0,1.0,468.194444,3.0,0.0,0.000000,no,0.0,no,0.0,no,0.0,no,29.0,0.420290,yes,52.0,yes,1816.0,yes,371.0,yes,0.96,58,72,1.24,36974,637.48,0,8,71,1.06


In [6]:
df.DupClaimNum_IP.unique()

array([ 0.,  1.,  3.,  2.,  7., 10.,  9.,  4.,  6., 11.,  5.,  8., 22.,
       13., 15., 19.])

In [7]:
df.DupClaimMultiState_IP.unique()

array(['no', 'yes'], dtype=object)

In [36]:
encode_cols

['Provider',
 'PotentialFraud',
 'Serves',
 'DupClaim_IP',
 'DupClaimMultiState_IP',
 'DupClaimMultiProvider_IP',
 'DupClaimMultiStartDt_IP',
 'DupClaim_OP',
 'DupClaimMultiState_OP',
 'DupClaimMultiProvider_OP',
 'DupClaimMultiStartDt_OP']

In [13]:
df.head()

,PotentialFraud,DurationMedianIP,DurationMeanIP,DurationMedianOP,DurationMeanOP,ChronCondsMedian,ChronCondsMean,ChronConsMode,PtAgeMedian,PtAgeMean,NumPtsIP,NumPtsOP,NumStatesIP,NumStatesOP,SharePhysician,MergeAvgClaimReimbursed,NumTopCodeClaim,DupClaimNum_IP,DupClaimRatio_IP,DupClaim_IP,DupClaimStatesNum_IP,DupClaimMultiState_IP,DupClaimProviderNum_IP,DupClaimMultiProvider_IP,DupClaimStartDtNum_IP,DupClaimMultiStartDt_IP,DupClaimNum_OP,DupClaimRatio_OP,DupClaim_OP,DupClaimStatesNum_OP,DupClaimMultiState_OP,DupClaimProviderNum_OP,DupClaimMultiProvider_OP,DupClaimStartDtNum_OP,DupClaimMultiStartDt_OP,AvgDuration,NumOfPt,NumOfClm,ClmPerPt,TotalRev,RevPerPt,ClmNoPhy,ClmAllPhy,ClmNoProc,ClmPerDt,Serves__Both,Serves__IP,Serves__OP
Provider,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
PRV51001,No,4.0,5.000000,0.0,0.550000,6.0,5.760000,5.0,79.0,77.880000,5.0,19.0,1.0,1.0,1.0,4185.600000,5.0,0.0,0.000000,0,0.0,0,0.0,0,0.0,0,11.0,0.550000,1,50.0,1,422.0,1,278.0,1,1.44,24,25,1.04,109980,4582.50,0,2,23,1.09,1,0,0
PRV51003,Yes,4.0,5.161290,0.0,2.357143,7.0,6.674242,6.0,71.0,69.083333,53.0,66.0,3.0,3.0,1.0,4588.409091,25.0,1.0,0.016129,1,2.0,1,2.0,1,2.0,1,25.0,0.357143,1,52.0,1,1351.0,1,372.0,1,3.67,117,132,1.13,671956,5743.21,0,4,93,1.19,1,0,0
PRV51004,No,0.0,0.000000,0.0,1.429530,7.0,6.812081,8.0,72.0,71.261745,0.0,138.0,0.0,9.0,1.0,350.134228,5.0,0.0,0.000000,0,0.0,0,0.0,0,0.0,0,72.0,0.483221,1,52.0,1,3006.0,1,375.0,1,1.43,138,149,1.08,52480,380.29,0,12,149,1.27,0,0,1
PRV51005,Yes,0.0,0.000000,0.0,1.088412,7.0,6.886695,5.0,70.0,69.526180,0.0,495.0,0.0,4.0,1.0,241.124464,25.0,0.0,0.000000,0,0.0,0,0.0,0,0.0,0,530.0,0.454936,1,52.0,1,3920.0,1,382.0,1,1.09,495,1165,2.35,284610,574.97,2,98,1165,3.27,0,0,1
PRV51007,No,5.0,5.333333,0.0,0.768116,7.0,6.986111,6.0,69.5,68.305556,3.0,56.0,1.0,2.0,1.0,468.194444,3.0,0.0,0.000000,0,0.0,0,0.0,0,0.0,0,29.0,0.420290,1,52.0,1,1816.0,1,371.0,1,0.96,58,72,1.24,36974,637.48,0,8,71,1.06,1,0,0


In [104]:
encode_cols = df[~df.applymap(np.isreal)].dropna(axis=1).columns.to_list()
encode_cols
df[encode_cols[3:]] = df[encode_cols[3:]].applymap(lambda x: 1 if x.lower() == 'yes' else 0)
df = pd.concat([df,pd.get_dummies(df['Serves'],prefix='Serves_')],axis=1)
df.drop('Serves', axis=1, inplace=True)
df.set_index('Provider',inplace=True)
target_df = df[['PotentialFraud']]
target_df['PotentialFraud'] = target_df.PotentialFraud.map(lambda x: 1 if x.lower() == 'yes' else 0)
df.drop('PotentialFraud',axis=1, inplace=True)
df.head()

C:\Users\patri\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,DurationMedianIP,DurationMeanIP,DurationMedianOP,DurationMeanOP,ChronCondsMedian,ChronCondsMean,ChronConsMode,PtAgeMedian,PtAgeMean,NumPtsIP,NumPtsOP,NumStatesIP,NumStatesOP,SharePhysician,MergeAvgClaimReimbursed,NumTopCodeClaim,DupClaimNum_IP,DupClaimRatio_IP,DupClaim_IP,DupClaimStatesNum_IP,DupClaimMultiState_IP,DupClaimProviderNum_IP,DupClaimMultiProvider_IP,DupClaimStartDtNum_IP,DupClaimMultiStartDt_IP,DupClaimNum_OP,DupClaimRatio_OP,DupClaim_OP,DupClaimStatesNum_OP,DupClaimMultiState_OP,DupClaimProviderNum_OP,DupClaimMultiProvider_OP,DupClaimStartDtNum_OP,DupClaimMultiStartDt_OP,AvgDuration,NumOfPt,NumOfClm,ClmPerPt,TotalRev,RevPerPt,ClmNoPhy,ClmAllPhy,ClmNoProc,ClmPerDt,Serves__Both,Serves__IP,Serves__OP
Provider,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
PRV51001,4.0,5.000000,0.0,0.550000,6.0,5.760000,5.0,79.0,77.880000,5.0,19.0,1.0,1.0,1.0,4185.600000,5.0,0.0,0.000000,0,0.0,0,0.0,0,0.0,0,11.0,0.550000,1,50.0,1,422.0,1,278.0,1,1.44,24,25,1.04,109980,4582.50,0,2,23,1.09,1,0,0
PRV51003,4.0,5.161290,0.0,2.357143,7.0,6.674242,6.0,71.0,69.083333,53.0,66.0,3.0,3.0,1.0,4588.409091,25.0,1.0,0.016129,1,2.0,1,2.0,1,2.0,1,25.0,0.357143,1,52.0,1,1351.0,1,372.0,1,3.67,117,132,1.13,671956,5743.21,0,4,93,1.19,1,0,0
PRV51004,0.0,0.000000,0.0,1.429530,7.0,6.812081,8.0,72.0,71.261745,0.0,138.0,0.0,9.0,1.0,350.134228,5.0,0.0,0.000000,0,0.0,0,0.0,0,0.0,0,72.0,0.483221,1,52.0,1,3006.0,1,375.0,1,1.43,138,149,1.08,52480,380.29,0,12,149,1.27,0,0,1
PRV51005,0.0,0.000000,0.0,1.088412,7.0,6.886695,5.0,70.0,69.526180,0.0,495.0,0.0,4.0,1.0,241.124464,25.0,0.0,0.000000,0,0.0,0,0.0,0,0.0,0,530.0,0.454936,1,52.0,1,3920.0,1,382.0,1,1.09,495,1165,2.35,284610,574.97,2,98,1165,3.27,0,0,1
PRV51007,5.0,5.333333,0.0,0.768116,7.0,6.986111,6.0,69.5,68.305556,3.0,56.0,1.0,2.0,1.0,468.194444,3.0,0.0,0.000000,0,0.0,0,0.0,0,0.0,0,29.0,0.420290,1,52.0,1,1816.0,1,371.0,1,0.96,58,72,1.24,36974,637.48,0,8,71,1.06,1,0,0


In [45]:
#target_df

In [17]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import model_selection

In [105]:
sm = SMOTE(random_state=0)
X_train, X_test, y_train,y_test = train_test_split(df,target_df, random_state=0,test_size=0.2, stratify=y)
X_train, y_train = sm.fit_sample(X_train,y_train)

NameError: name 'y' is not defined

In [108]:
X_train,X_test,y_train,y_test = train_test_split(df,target_df.PotentialFraud, random_state=0,test_size=0.2,stratify=target_df.PotentialFraud)

##### Linear Model

In [28]:
logit = LogisticRegression(penalty='l1',C=0.1,random_state=0, solver='liblinear')

In [78]:
logit.fit(df,target_df)

C:\Users\patri\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


LogisticRegression(C=0.1, penalty='l1', random_state=0, solver='liblinear')

In [79]:
logit.score(X_train,y_train)

0.7326859187324304

In [90]:
logit.score(X_test,y_test)

0.9473197781885397

In [30]:
l_gs = {'C':np.linspace(0,1,20)}

In [31]:
logit_gs = GridSearchCV(estimator= logit, param_grid= l_gs, cv=5, scoring='recall')

In [32]:
logit_gs.fit(X_train,y_train)

C:\Users\patri\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\patri\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\patri\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\patri\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1360, in fit
    sample_weight=sample_weight)
  File "C:\Users\patri\Anaconda3\lib\site-packages\sklearn\svm\_base.py", line 969, in _fit_liblinear
    epsilon, sample_weight)
  File "sklearn\svm\_liblinear.pyx", line 52, in sklearn.s

C:\Users\patri\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\patri\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\patri\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\patri\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**

C:\Users\patri\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\patri\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\patri\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\patri\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**

C:\Users\patri\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\patri\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\patri\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)
C:\Users\patri\Anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**

GridSearchCV(cv=5,
             estimator=LogisticRegression(C=0.1, penalty='l1', random_state=0,
                                          solver='liblinear'),
             param_grid={'C': array([0.        , 0.05263158, 0.10526316, 0.15789474, 0.21052632,
       0.26315789, 0.31578947, 0.36842105, 0.42105263, 0.47368421,
       0.52631579, 0.57894737, 0.63157895, 0.68421053, 0.73684211,
       0.78947368, 0.84210526, 0.89473684, 0.94736842, 1.        ])},
             scoring='recall')

In [33]:
logit_gs.best_score_

0.922318905906524

In [139]:
logit_gs.best_estimator_

LogisticRegression(C=0.6842105263157894, penalty='l1', random_state=0,
                   solver='liblinear')

In [130]:
logit_gs.best_score_

0.9150312348263213

In [85]:
prop = logit.predict_proba(X_train)

In [89]:
(prop > 0.9).map(lambda x: 1 if x else 0)

AttributeError: 'numpy.ndarray' object has no attribute 'map'

In [93]:
np.linspace(0,1,20)

array([0.        , 0.05263158, 0.10526316, 0.15789474, 0.21052632,
       0.26315789, 0.31578947, 0.36842105, 0.42105263, 0.47368421,
       0.52631579, 0.57894737, 0.63157895, 0.68421053, 0.73684211,
       0.78947368, 0.84210526, 0.89473684, 0.94736842, 1.        ])

##### Tree Model 

In [54]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

In [109]:
def score(cm):
    print(cm[0][0]/(cm[0][0] + cm[0][1]))
    print(cm[1][1]/(cm[1][1] + cm[1][0]))

In [79]:
rfc_gs_params = {'min_samples_split':np.arange(5,21,5),'min_samples_leaf':np.arange(5,20,5), 'max_depth': np.arange(11,20,1)}
rfc_gs_params

{'min_samples_split': array([ 5, 10, 15, 20]),
 'min_samples_leaf': array([ 5, 10, 15]),
 'max_depth': array([11, 12, 13, 14, 15, 16, 17, 18, 19])}

In [84]:
# rfc = RandomForestClassifier(min_samples_split=4,min_samples_leaf=4,criterion='gini' )
# rfc.fit(X_train,y_train)
# rfc.score(X_train,y_train)


In [57]:
rfc.classes_

array([0, 1], dtype=int64)

In [63]:
print(confusion_matrix(y_train,rfc.predict(X_train)))
cm = confusion_matrix(y_train,rfc.predict(X_train))


[[3856   57]
 [  35 3878]]
0.9854331714796831
0.9910554561717353


In [ ]:
# pd.DataFrame(rfc.feature_importances_.reshape(1,-1),columns=X_train.columns.to_list())
# pd.Series(rfc.feature_importances_,index=X_train.columns.to_list()).sort_values(ascending=False).head(20)

In [112]:
rfc_gs = GridSearchCV(rfc, param_grid=rfc_gs_params)
rfc_gs.fit(X_train,y_train)
rfc_gs.best_score_
print(confusion_matrix(y_train,rfc.predict(X_train)))
cm = confusion_matrix(y_train,rfc.predict(X_train))
# score_(cm)

[[3829   94]
 [  41  364]]


In [86]:
score(cm)

0.9859442882698697
0.990799897776642


In [89]:
rfc_gs.best_estimator_.score(X_train,y_train)

0.9833887043189369

In [87]:
rfc_gs.best_score_

0.9465927313233692